In [1]:
import sys
import numpy as np
import os
import flopy 
import shapely
from shapely.geometry import (LineString, MultiLineString, MultiPoint,
                              MultiPolygon, Point, Polygon)

# Exe
# init paths
exe_name = "C:/Users/josej/Downloads/MODFLOW/mf6.3.0/bin/mf6.exe"

print(sys.version)
print("numpy version: {}".format(np.__version__))
print("flopy version: {}".format(flopy.__version__))


3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.21.5
flopy version: 3.3.6


In [2]:
os.getcwd()

'C:\\Users\\josej\\Jupyter Lab\\Temporal'

In [3]:
# Create simulation
model_name = "model"
sim_name = model_name + ".sim"

model_ws = os.path.join(os.getcwd(), model_name)
# make the directory if it does not exist
if not os.path.isdir( model_ws ):
    os.makedirs( model_ws, exist_ok=True)
else:
    pass

os.chdir( model_ws )

# SIM
sim = flopy.mf6.MFSimulation( 
    sim_name= sim_name, version="mf6", exe_name = exe_name, sim_ws=model_ws
)

In [4]:
# GWF
model_nam_file = "{}.nam".format(model_name)
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name, model_nam_file=model_nam_file)

# DIS
Nlay = 1
nrow = 10
ncol = 10
delc = 10 # meters
delr = 10 # meters
top = 0
bot_array = -20


# DIS package
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay = Nlay,
    nrow = nrow,
    ncol = ncol,
    delr = delr,
    delc = delc,
    top = top,
    botm = bot_array
)

# TDIS 
tdis_rc = [ (1.0, 1, 1.0) ]
nper = len( tdis_rc )
tdis_package = flopy.mf6.modflow.mftdis.ModflowTdis(
    sim, time_units="DAYS", nper= nper, perioddata=tdis_rc
)

# IMS 
# Sometimes IMS package gives error that disappears when the cell is run again.
ims_package = flopy.mf6.modflow.mfims.ModflowIms(
    sim,
    filename = "{}.ims".format(model_name),
    complexity = "SIMPLE"
)
sim.register_ims_package(ims_package, [gwf.name])

# IC
strt = 0
ic_package = flopy.mf6.modflow.mfgwfic.ModflowGwfic(
    gwf, strt= strt
)

# OC
oc_package = flopy.mf6.modflow.mfgwfoc.ModflowGwfoc(
    gwf
)


In [5]:
# CHD
chd_stpd = flopy.mf6.ModflowGwfchd.stress_period_data.empty(gwf,
                                                          maxbound = 1,
                                                          boundnames=True,
                                                          timeseries=False,
                                                          stress_periods= range( 1 )
                                                         )

#chd_stpd = np.recarray( [(0, 5, 5), -5, ""], dtype= chd_stpd[0].dtype)
chd_stpd[0][0] = ( (0, 5, 5), -5, "")

chd = flopy.mf6.modflow.mfgwfchd.ModflowGwfchd(gwf,
                                               maxbound=1,
                                               stress_period_data= chd_stpd)

In [6]:
# NPF
npf_package = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(
    gwf,
    save_flows=True,
    save_specific_discharge=True,
    icelltype= 1,
    k = 0.02,
    k33 = 0.002
)

# OC 
oc_package = flopy.mf6.modflow.mfgwfoc.ModflowGwfoc(
    gwf,
    budget_filerecord= model_name + ".cbc",
    head_filerecord= model_name + ".hds",
    headprintrecord=["COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL"],
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("HEAD", "FIRST"), ("HEAD", "LAST"), ("BUDGET", "LAST")],
)



# OBS
# obs_ws is folder
obs_ws = "observations"
# make the directory if it does not exist
if not os.path.isdir( obs_ws ):
    os.makedirs( obs_ws, exist_ok=True)
else:
    pass


chd_obs = {
    (os.path.join(obs_ws,'chd_obs_binary.csv'), "binary"): [
        ('chd-(0, 5, 5)', "CHD", (0, 5,5)),
       
    ],
    os.path.join(obs_ws,'chdws.csv'): [
        ('chd-(0, 5, 5)', "CHD", (0, 5,5)),
    ],
}
# initialize obs package
#gwf.remove_package("OBS")
chd.obs.initialize(
    pname="obs",
    filename="test.ghb.obs",
    digits=9, 
    print_input=True,
    continuous= chd_obs,
)

In [7]:
sim.write_simulation(silent = False)
sim.run_simulation(silent = False, )

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package ims_-1...
  writing model model...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package chd_0...
    writing package npf...
    writing package oc...
    writing package obs...
FloPy is using the following executable to run the model: C:/Users/josej/Downloads/MODFLOW/mf6.3.0/bin/mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.3.0 03/04/2022

   MODFLOW 6 compiled Mar 02 2022 15:29:04 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.5.0
                             Build 20211109_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to 

(True, [])

In [8]:
new_ws = "model_copy"
# make the directory if it does not exist
if not os.path.isdir( new_ws ):
    os.makedirs( new_ws, exist_ok=True)
else:
    pass

# obs_ws is folder
obs_ws = os.path.join(new_ws,"observations")
# make the directory if it does not exist
if not os.path.isdir( obs_ws ):
    os.makedirs( obs_ws, exist_ok=True)
else:
    pass


In [9]:
sim_load = flopy.mf6.mfsimulation.MFSimulation.load(
    sim_name='sim',
    version='mf6',
    exe_name=exe_name,
    sim_ws= model_ws,
    verbosity_level=2,
    write_headers=True)

# Change model_ws
sim_load.set_sim_path(new_ws)

loading simulation...
  loading simulation name file...
      loading block options...
      loading block timing...
        loading data tdis6...
      loading block models...
        loading data models...
      loading block exchanges...
      loading block solutiongroup...
        loading data solutiongroup...
  loading tdis package...
      loading block options...
        loading data time_units...
      loading block dimensions...
        loading data nper...
      loading block perioddata...
        loading data perioddata...
  loading model gwf6...
      loading block options...
      loading block packages...
        loading data packages...
    loading package dis...
      loading block options...
      loading block dimensions...
        loading data nlay...
        loading data nrow...
        loading data ncol...
      loading block griddata...
        loading data delr...
        loading data delc...
        loading data top...
        loading data botm...
    loading pa

In [10]:
sim_load.write_simulation(silent = False)
sim_load.run_simulation(silent = False, )

writing simulation...
  writing simulation name file...
      writing block options...
        writing data continue...
        writing data nocheck...
        writing data memory_print_option...
        writing data maxerrors...
      writing block timing...
        writing data tdis6...
      writing block models...
        writing data models...
      writing block exchanges...
        writing data exchanges...
      writing block solutiongroup...
        writing data mxiter (0)...
        writing data solutiongroup (0)...
  writing simulation tdis package...
      writing block options...
        writing data time_units...
        writing data start_date_time...
        writing data ats_filerecord...
      writing block dimensions...
        writing data nper...
      writing block perioddata...
        writing data perioddata...
  writing ims package model...
      writing block options...
        writing data print_option...
        writing data complexity...
        writing data

(False, [])

In [11]:
os.chdir( model_ws )

In [12]:
os.getcwd()

'C:\\Users\\josej\\Jupyter Lab\\Temporal\\model'